# Module 3: Filtering & Enhancement

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/03_Filtering_Enhancement.ipynb)

**Week 5-6: Image Filtering & Enhancement Techniques**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Topics
- Convolution fundamentals
- Gaussian Blur
- Median Filter
- Bilateral Filter
- Image Sharpening

**Key Functions**: `cv2.filter2D()`, `cv2.GaussianBlur()`, `cv2.medianBlur()`, `cv2.bilateralFilter()`

---
## Mathematical Foundations: Signal Processing and Convolution Theory

Before working with filters, let's establish the **theoretical mathematical foundations** of image filtering.

### Topics Covered
1. **Convolution Theory**: Mathematical definition and properties
2. **Linear Systems**: Shift-invariance and impulse response
3. **Kernels as Discrete Filters**: Mathematical representation
4. **Frequency Domain**: Fourier analysis of filtering
5. **Statistical Filters**: Order statistics and robust estimation
6. **Edge-Preserving Filters**: Bilateral filtering mathematics

### 1. Convolution: Mathematical Definition

#### 2D Discrete Convolution

The **convolution** of an image $f$ with a kernel $h$ is defined as:

$$
g(x, y) = (f \ast h)(x, y) = \sum_{i=-\infty}^{\infty} \sum_{j=-\infty}^{\infty} f(i, j) \cdot h(x-i, y-j)
$$

For **finite kernels** of size $(2k+1) \times (2k+1)$:

$$
g(x, y) = \sum_{i=-k}^{k} \sum_{j=-k}^{k} f(x+i, y+j) \cdot h(i, j)
$$

**Alternative form** (correlation - what OpenCV uses):

$$
g(x, y) = \sum_{i=-k}^{k} \sum_{j=-k}^{k} f(x-i, y-j) \cdot h(i, j)
$$

**Matrix notation**: For a 3×3 kernel centered at pixel $(x, y)$:

$$
g(x, y) = \begin{bmatrix}
f(x-1, y-1) & f(x, y-1) & f(x+1, y-1) \\
f(x-1, y) & f(x, y) & f(x+1, y) \\
f(x-1, y+1) & f(x, y+1) & f(x+1, y+1)
\end{bmatrix}
\odot
\begin{bmatrix}
h_{-1,-1} & h_{0,-1} & h_{1,-1} \\
h_{-1,0} & h_{0,0} & h_{1,0} \\
h_{-1,1} & h_{0,1} & h_{1,1}
\end{bmatrix}
$$

Where $\odot$ denotes element-wise multiplication followed by summation.

#### Properties of Convolution

1. **Commutative**: $f \ast h = h \ast f$

2. **Associative**: $(f \ast h_1) \ast h_2 = f \ast (h_1 \ast h_2)$

3. **Distributive**: $f \ast (h_1 + h_2) = f \ast h_1 + f \ast h_2$

4. **Linearity**: $\alpha f \ast h = \alpha(f \ast h)$

5. **Identity**: $f \ast \delta = f$, where $\delta$ is the impulse function:
   $$
   \delta(x, y) = \begin{cases} 1 & \text{if } (x, y) = (0, 0) \\ 0 & \text{otherwise} \end{cases}
   $$

In [ ]:
# Demonstrate convolution mathematics
print("=" * 70)
print("2D CONVOLUTION: MATHEMATICAL DEMONSTRATION")
print("=" * 70)

# Create a simple 5x5 image
f = np.array([
    [10, 10, 10, 10, 10],
    [10, 50, 50, 50, 10],
    [10, 50, 100, 50, 10],
    [10, 50, 50, 50, 10],
    [10, 10, 10, 10, 10]
], dtype=np.float32)

# Define a 3x3 averaging kernel
h = np.ones((3, 3), dtype=np.float32) / 9.0

print("\nInput image f (5×5):")
print(f.astype(int))

print("\nKernel h (3×3 averaging):")
print(h)
print(f"\nKernel sum: {h.sum():.4f} (should be 1.0 for brightness preservation)")

# Manual convolution at center pixel (2, 2)
print("\n" + "="*70)
print("MANUAL CONVOLUTION at pixel (2, 2)")
print("="*70)

x, y = 2, 2
neighborhood = f[y-1:y+2, x-1:x+2]
print(f"\nNeighborhood at ({x}, {y}):")
print(neighborhood.astype(int))

# Element-wise multiplication
element_wise = neighborhood * h
print("\nElement-wise multiplication (f ⊙ h):")
print(element_wise)

# Sum to get convolution result
result_manual = element_wise.sum()
print(f"\nConvolution result: Σ(f ⊙ h) = {result_manual:.2f}")

# Mathematical formula
print("\nMathematical expression:")
print("g(2,2) = Σᵢ Σⱼ f(2+i, 2+j) · h(i,j)")
formula_str = ""
for i in range(-1, 2):
    for j in range(-1, 2):
        val_f = f[y+j, x+i]
        val_h = h[j+1, i+1]
        if i == -1 and j == -1:
            formula_str += f"{val_f:.0f}×{val_h:.4f}"
        else:
            formula_str += f" + {val_f:.0f}×{val_h:.4f}"
print(f"      = {formula_str}")
print(f"      = {result_manual:.2f}")

# Verify with OpenCV
print("\n" + "="*70)
print("VERIFICATION WITH OpenCV")
print("="*70)

g = cv2.filter2D(f, -1, h)
print("\nConvolved image g = f ∗ h:")
print(g.astype(int))
print(f"\nValue at (2,2): {g[2, 2]:.2f}")
print(f"Manual calculation: {result_manual:.2f}")
print(f"Match: {np.isclose(g[2, 2], result_manual)}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(f, cmap='gray', interpolation='nearest', vmin=0, vmax=100)
axes[0].set_title('Input f (5×5)', fontsize=12)
for i in range(5):
    for j in range(5):
        axes[0].text(j, i, f'{int(f[i,j])}', ha='center', va='center',
                    color='red' if (i==2 and j==2) else 'white', fontweight='bold')
axes[0].axis('off')

axes[1].imshow(h, cmap='gray', interpolation='nearest')
axes[1].set_title('Kernel h (3×3)\nAveraging Filter', fontsize=12)
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, f'{h[i,j]:.3f}', ha='center', va='center',
                    color='white', fontsize=10)
axes[1].axis('off')

axes[2].imshow(g, cmap='gray', interpolation='nearest', vmin=0, vmax=100)
axes[2].set_title('Output g = f ∗ h (5×5)', fontsize=12)
for i in range(5):
    for j in range(5):
        axes[2].text(j, i, f'{int(g[i,j])}', ha='center', va='center',
                    color='red' if (i==2 and j==2) else 'white', fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Convolution is a weighted sum over the neighborhood!")

### 2. Common Kernels and Their Mathematical Properties

#### Box Blur (Averaging) Filter

**Definition**: Uniform averaging of neighborhood

$$
h_{\text{box}}(i, j) = \frac{1}{(2k+1)^2}, \quad i, j \in [-k, k]
$$

For $k=1$ (3×3 kernel):

$$
h_{\text{box}} = \frac{1}{9} \begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

**Properties**:
- $\sum_{i,j} h(i,j) = 1$ (preserves brightness)
- Separable: $h_{\text{box}} = h_{1D} \otimes h_{1D}^T$ where $h_{1D} = \frac{1}{3}[1, 1, 1]^T$

#### Gaussian Filter

**Definition**: Weights follow a 2D Gaussian distribution

$$
G(x, y; \sigma) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

**Discrete approximation**:

$$
h_G(i, j) = \frac{G(i, j; \sigma)}{\sum_{m,n} G(m, n; \sigma)}
$$

**Example** (5×5 Gaussian with $\sigma = 1$):

$$
h_G \approx \frac{1}{273} \begin{bmatrix}
1 & 4 & 7 & 4 & 1 \\
4 & 16 & 26 & 16 & 4 \\
7 & 26 & 41 & 26 & 7 \\
4 & 16 & 26 & 16 & 4 \\
1 & 4 & 7 & 4 & 1
\end{bmatrix}
$$

**Properties**:
- **Separable**: $G(x, y) = G(x) \cdot G(y)$
- **Self-convolving**: $G_{\sigma_1} \ast G_{\sigma_2} = G_{\sqrt{\sigma_1^2 + \sigma_2^2}}$
- **Optimal smoothing**: Minimizes uncertainty in spatial-frequency domain

#### Gradient Kernels (Edge Detection)

**Sobel operator** (approximates derivative):

Horizontal gradient:
$$
S_x = \begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix}
= \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix}
\begin{bmatrix} -1 & 0 & 1 \end{bmatrix}
$$

Vertical gradient:
$$
S_y = \begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1
\end{bmatrix}
$$

**Gradient magnitude**:
$$
|\nabla f| = \sqrt{(f \ast S_x)^2 + (f \ast S_y)^2}
$$

**Gradient direction**:
$$
\theta = \arctan\left(\frac{f \ast S_y}{f \ast S_x}\right)
$$

#### Laplacian (Second Derivative)

**Definition**: Sum of second derivatives

$$
\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}
$$

**Discrete approximation**:

$$
L = \begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{bmatrix}
\quad \text{or} \quad
L = \begin{bmatrix}
1 & 1 & 1 \\
1 & -8 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

**Properties**:
- $\sum_{i,j} L(i,j) = 0$ (zero-sum filter)
- Detects edges in all directions
- Sensitive to noise

In [ ]:
# Demonstrate common kernels mathematically
print("=" * 70)
print("COMMON KERNELS: MATHEMATICAL PROPERTIES")
print("=" * 70)

# Box blur kernel
box_3x3 = np.ones((3, 3), dtype=np.float32) / 9
print("\nBOX BLUR (3×3):")
print(box_3x3)
print(f"Sum: {box_3x3.sum():.4f}")
print(f"Mean: {box_3x3.mean():.4f}")

# Gaussian kernel (manual calculation)
print("\n" + "="*70)
print("GAUSSIAN KERNEL (σ=1)")
print("="*70)

sigma = 1.0
size = 5
center = size // 2

# Create Gaussian kernel manually
gaussian_manual = np.zeros((size, size), dtype=np.float32)
print(f"\nFormula: G(x,y) = (1/(2πσ²)) · exp(-(x²+y²)/(2σ²))")
print(f"With σ = {sigma}:\n")

for i in range(size):
    for j in range(size):
        x = j - center
        y = i - center
        gaussian_manual[i, j] = np.exp(-(x**2 + y**2) / (2 * sigma**2)) / (2 * np.pi * sigma**2)

# Normalize
gaussian_manual /= gaussian_manual.sum()

print("Normalized Gaussian kernel (5×5):")
print(gaussian_manual)
print(f"\nSum: {gaussian_manual.sum():.6f}")
print(f"Center weight: {gaussian_manual[center, center]:.6f}")

# Sobel kernels
print("\n" + "="*70)
print("SOBEL KERNELS (Gradient Detection)")
print("="*70)

sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.float32)

sobel_y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]], dtype=np.float32)

print("\nSobel X (horizontal edges):")
print(sobel_x.astype(int))
print(f"Sum: {sobel_x.sum():.1f} (zero-sum filter)")

print("\nSobel Y (vertical edges):")
print(sobel_y.astype(int))
print(f"Sum: {sobel_y.sum():.1f} (zero-sum filter)")

# Verify separability
smooth = np.array([[1], [2], [1]], dtype=np.float32)
diff = np.array([[-1, 0, 1]], dtype=np.float32)
sobel_x_sep = smooth @ diff
print(f"\nSeparability check (Sobel X):")
print(f"  [1,2,1]ᵀ ⊗ [-1,0,1] = \n{sobel_x_sep.astype(int)}")
print(f"  Matches Sobel X: {np.allclose(sobel_x, sobel_x_sep)}")

# Laplacian
print("\n" + "="*70)
print("LAPLACIAN (Second Derivative)")
print("="*70)

laplacian_4 = np.array([[ 0,  1,  0],
                        [ 1, -4,  1],
                        [ 0,  1,  0]], dtype=np.float32)

laplacian_8 = np.array([[ 1,  1,  1],
                        [ 1, -8,  1],
                        [ 1,  1,  1]], dtype=np.float32)

print("\nLaplacian (4-neighbor):")
print(laplacian_4.astype(int))
print(f"Sum: {laplacian_4.sum():.1f} (zero-sum)")

print("\nLaplacian (8-neighbor):")
print(laplacian_8.astype(int))
print(f"Sum: {laplacian_8.sum():.1f} (zero-sum)")

# Visualize kernels
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

kernels = [
    (box_3x3, 'Box Blur', 'gray'),
    (gaussian_manual, 'Gaussian', 'hot'),
    (sobel_x, 'Sobel X', 'RdBu'),
    (sobel_y, 'Sobel Y', 'RdBu'),
    (laplacian_4, 'Laplacian-4', 'RdBu'),
    (laplacian_8, 'Laplacian-8', 'RdBu')
]

for idx, (kernel, title, cmap) in enumerate(kernels):
    ax = axes[idx // 3, idx % 3]
    im = ax.imshow(kernel, cmap=cmap, interpolation='nearest')
    ax.set_title(f'{title}\nΣ={kernel.sum():.2f}', fontsize=12)
    plt.colorbar(im, ax=ax, fraction=0.046)
    
    # Add values as text
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            ax.text(j, i, f'{kernel[i,j]:.2f}', ha='center', va='center',
                   color='white' if abs(kernel[i,j]) > 0.1 else 'black',
                   fontsize=9)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Different kernels detect different features!")
print("  - Sum = 1: Preserves brightness (smoothing)")
print("  - Sum = 0: Detects changes (edges, derivatives)")

### 3. Non-Linear Filters: Order Statistics

#### Median Filter

Unlike convolution (linear operation), the **median filter** is a **non-linear** filter based on **order statistics**.

**Definition**:

$$
g(x, y) = \text{median}\{f(x+i, y+j) : (i,j) \in W\}
$$

Where $W$ is the window (neighborhood).

**For window size $n$**:
1. Sort pixel values: $v_1 \leq v_2 \leq \cdots \leq v_n$
2. Return middle value:

$$
\text{median} = \begin{cases}
v_{(n+1)/2} & \text{if } n \text{ is odd} \\
\frac{v_{n/2} + v_{n/2+1}}{2} & \text{if } n \text{ is even}
\end{cases}
$$

**Properties**:
- **Edge-preserving**: Does not blur edges like linear filters
- **Robust to outliers**: Immune to salt-and-pepper noise
- **Not linear**: median$(a + b) \neq$ median$(a)$ + median$(b)$
- **Computational cost**: $O(n \log n)$ per pixel (sorting)

#### General Order-Statistic Filters

**Minimum filter** (erosion):
$$
g(x, y) = \min_{(i,j) \in W} f(x+i, y+j)
$$

**Maximum filter** (dilation):
$$
g(x, y) = \max_{(i,j) \in W} f(x+i, y+j)
$$

**Percentile filter**:
$$
g(x, y) = p\text{-th percentile of } \{f(x+i, y+j) : (i,j) \in W\}
$$

In [ ]:
# Demonstrate median filter mathematics
print("=" * 70)
print("MEDIAN FILTER: ORDER STATISTICS")
print("=" * 70)

# Create a simple example with outlier
neighborhood = np.array([10, 12, 15, 255, 11, 13, 14, 12, 16])  # 255 is outlier

print("\nNeighborhood values (3×3 window flattened):")
print(neighborhood)

# Sort values
sorted_values = np.sort(neighborhood)
print("\nSorted values:")
print(sorted_values)

# Calculate median
n = len(neighborhood)
if n % 2 == 1:
    median = sorted_values[n // 2]
    print(f"\nMedian (n={n}, odd):")
    print(f"  Position: {n//2} (0-indexed)")
    print(f"  Value: {median}")
else:
    median = (sorted_values[n//2 - 1] + sorted_values[n//2]) / 2
    print(f"\nMedian (n={n}, even):")
    print(f"  Average of positions {n//2-1} and {n//2}")
    print(f"  Value: {median}")

# Calculate mean for comparison
mean = neighborhood.mean()
print(f"\nMean (for comparison): {mean:.2f}")
print(f"Median: {median:.2f}")
print(f"\nNotice: Median ({median:.2f}) is robust to outlier (255)")
print(f"        Mean ({mean:.2f}) is heavily affected by outlier")

# Demonstrate on noisy image
print("\n" + "="*70)
print("MEDIAN FILTER ON SALT-AND-PEPPER NOISE")
print("="*70)

# Create a small test image
clean = np.full((7, 7), 100, dtype=np.uint8)
noisy = clean.copy()

# Add salt and pepper
np.random.seed(42)
salt_coords = [(1, 1), (2, 4), (4, 2), (5, 5)]
pepper_coords = [(1, 5), (3, 3), (5, 1)]

for coord in salt_coords:
    noisy[coord] = 255
for coord in pepper_coords:
    noisy[coord] = 0

# Apply median filter
denoised = cv2.medianBlur(noisy, 3)

print("\nClean image (7×7, all values = 100):")
print(clean)

print("\nNoisy image (salt=255, pepper=0):")
print(noisy)

print("\nMedian filtered (3×3 window):")
print(denoised)

# Calculate restoration quality
mse_noisy = np.mean((clean - noisy) ** 2)
mse_denoised = np.mean((clean - denoised) ** 2)

print(f"\nMean Squared Error:")
print(f"  Noisy vs Clean: {mse_noisy:.2f}")
print(f"  Denoised vs Clean: {mse_denoised:.2f}")
print(f"  Improvement: {(1 - mse_denoised/mse_noisy)*100:.1f}%")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(clean, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
axes[0].set_title('Clean Image', fontsize=12)
axes[0].axis('off')

axes[1].imshow(noisy, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
axes[1].set_title('Noisy (Salt & Pepper)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(denoised, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
axes[2].set_title('Median Filtered (3×3)', fontsize=12)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Median is the middle value after sorting - robust to outliers!")

### 4. Bilateral Filter: Edge-Preserving Smoothing

The **bilateral filter** is a **non-linear** filter that considers both spatial and intensity similarity.

#### Mathematical Definition

$$
g(x, y) = \frac{1}{W_p} \sum_{i,j \in \Omega} f(i, j) \cdot w(i, j, x, y)
$$

Where the **bilateral weight** is:

$$
w(i, j, x, y) = w_s(i, j, x, y) \cdot w_r(i, j, x, y)
$$

**Spatial weight** (distance in coordinate space):
$$
w_s(i, j, x, y) = \exp\left(-\frac{(i-x)^2 + (j-y)^2}{2\sigma_s^2}\right)
$$

**Range weight** (distance in intensity space):
$$
w_r(i, j, x, y) = \exp\left(-\frac{(f(i,j) - f(x,y))^2}{2\sigma_r^2}\right)
$$

**Normalization factor**:
$$
W_p = \sum_{i,j \in \Omega} w(i, j, x, y)
$$

#### Intuition

- **$\sigma_s$** (spatial sigma): Controls spatial extent
  - Larger $\sigma_s$ → wider spatial neighborhood
  
- **$\sigma_r$** (range sigma): Controls intensity similarity
  - Larger $\sigma_r$ → more tolerant to intensity differences
  - Smaller $\sigma_r$ → preserves edges better

#### Key Properties

1. **Edge preservation**: Pixels across edges have small $w_r$ → little influence
2. **Smooth regions**: Similar intensities → large $w_r$ → strong smoothing
3. **Non-linear**: Cannot be represented as convolution
4. **Computational cost**: $O(d^2)$ per pixel for window diameter $d$

#### Comparison with Gaussian

**Gaussian filter**: Only spatial weight
$$
g_{\text{Gauss}}(x, y) = \sum_{i,j} f(i, j) \cdot \exp\left(-\frac{(i-x)^2 + (j-y)^2}{2\sigma^2}\right)
$$

**Bilateral filter**: Spatial + Range weights
$$
g_{\text{Bilateral}}(x, y) = \sum_{i,j} f(i, j) \cdot \underbrace{\exp\left(-\frac{(i-x)^2 + (j-y)^2}{2\sigma_s^2}\right)}_{\text{spatial}} \cdot \underbrace{\exp\left(-\frac{(f(i,j) - f(x,y))^2}{2\sigma_r^2}\right)}_{\text{range}}
$$

In [ ]:
# Demonstrate bilateral filter mathematics
print("=" * 70)
print("BILATERAL FILTER: MATHEMATICAL DEMONSTRATION")
print("=" * 70)

# Create a simple step edge example
step_edge = np.array([
    [50, 50, 50, 150, 150, 150],
    [50, 50, 50, 150, 150, 150],
    [50, 50, 50, 150, 150, 150],
    [50, 50, 50, 150, 150, 150],
    [50, 50, 50, 150, 150, 150],
    [50, 50, 50, 150, 150, 150]
], dtype=np.uint8)

print("\nInput: Step edge image")
print(step_edge)

# Parameters
sigma_s = 1.5  # Spatial sigma
sigma_r = 30   # Range sigma

print(f"\nParameters:")
print(f"  σ_s (spatial) = {sigma_s}")
print(f"  σ_r (range) = {sigma_r}")

# Manual calculation at a specific pixel (2, 2) - left side
print("\n" + "="*70)
print("MANUAL CALCULATION at pixel (2, 2)")
print("="*70)

x, y = 2, 2
center_value = step_edge[y, x]
print(f"\nCenter pixel value: f(2,2) = {center_value}")

# Extract 3x3 neighborhood
neighborhood = step_edge[y-1:y+2, x-1:x+2].astype(float)
print(f"\nNeighborhood:")
print(neighborhood.astype(int))

# Calculate weights
print(f"\nWeight calculation:")
print(f"w(i,j) = exp(-((i-x)²+(j-y)²)/(2σ_s²)) · exp(-(f(i,j)-f(x,y))²/(2σ_r²))")
print(f"\nSpatial weights (distance-based):")

w_spatial = np.zeros((3, 3))
w_range = np.zeros((3, 3))
w_bilateral = np.zeros((3, 3))

for di in range(-1, 2):
    for dj in range(-1, 2):
        # Spatial weight
        spatial_dist = di**2 + dj**2
        w_spatial[di+1, dj+1] = np.exp(-spatial_dist / (2 * sigma_s**2))
        
        # Range weight
        intensity_diff = neighborhood[di+1, dj+1] - center_value
        w_range[di+1, dj+1] = np.exp(-(intensity_diff**2) / (2 * sigma_r**2))
        
        # Combined weight
        w_bilateral[di+1, dj+1] = w_spatial[di+1, dj+1] * w_range[di+1, dj+1]

print(w_spatial)
print(f"\nRange weights (intensity-based):")
print(w_range)
print(f"\nCombined bilateral weights:")
print(w_bilateral)

# Normalize and compute output
W_p = w_bilateral.sum()
w_normalized = w_bilateral / W_p
output_value = (neighborhood * w_normalized).sum()

print(f"\nNormalization factor W_p: {W_p:.6f}")
print(f"Normalized weights:")
print(w_normalized)
print(f"\nOutput value: {output_value:.2f}")
print(f"Input value: {center_value}")

# Now calculate at edge pixel (2, 3)
print("\n" + "="*70)
print("CALCULATION at EDGE pixel (2, 3)")
print("="*70)

x_edge, y_edge = 3, 2
center_value_edge = step_edge[y_edge, x_edge]
neighborhood_edge = step_edge[y_edge-1:y_edge+2, x_edge-1:x_edge+2].astype(float)

print(f"\nCenter pixel value: f(2,3) = {center_value_edge}")
print(f"\nNeighborhood (crosses edge!):")
print(neighborhood_edge.astype(int))

# Calculate bilateral weights for edge pixel
w_bilateral_edge = np.zeros((3, 3))
for di in range(-1, 2):
    for dj in range(-1, 2):
        spatial_dist = di**2 + dj**2
        w_s = np.exp(-spatial_dist / (2 * sigma_s**2))
        
        intensity_diff = neighborhood_edge[di+1, dj+1] - center_value_edge
        w_r = np.exp(-(intensity_diff**2) / (2 * sigma_r**2))
        
        w_bilateral_edge[di+1, dj+1] = w_s * w_r

print(f"\nBilateral weights at edge:")
print(w_bilateral_edge)
print(f"\nNotice: Weights for pixels with different intensity (left side) are smaller!")

# Visualize the concept
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(step_edge, cmap='gray', interpolation='nearest')
axes[0, 0].set_title('Input (Step Edge)', fontsize=12)
axes[0, 0].plot(x, y, 'ro', markersize=10, label='Pixel (2,2)')
axes[0, 0].plot(x_edge, y_edge, 'go', markersize=10, label='Pixel (2,3)')
axes[0, 0].legend()
axes[0, 0].axis('off')

axes[0, 1].imshow(w_spatial, cmap='hot', interpolation='nearest')
axes[0, 1].set_title('Spatial Weights\n(distance-based)', fontsize=12)
axes[0, 1].axis('off')

axes[0, 2].imshow(w_range, cmap='hot', interpolation='nearest')
axes[0, 2].set_title('Range Weights at (2,2)\n(intensity-based)', fontsize=12)
axes[0, 2].axis('off')

axes[1, 0].imshow(w_bilateral, cmap='hot', interpolation='nearest')
axes[1, 0].set_title('Bilateral Weights at (2,2)\n(spatial × range)', fontsize=12)
axes[1, 0].axis('off')

# Apply filters for comparison
gaussian = cv2.GaussianBlur(step_edge, (5, 5), sigma_s)
bilateral = cv2.bilateralFilter(step_edge, 5, sigma_r, sigma_s)

axes[1, 1].imshow(gaussian, cmap='gray', interpolation='nearest')
axes[1, 1].set_title(f'Gaussian Blur\n(edge blurred)', fontsize=12)
axes[1, 1].axis('off')

axes[1, 2].imshow(bilateral, cmap='gray', interpolation='nearest')
axes[1, 2].set_title(f'Bilateral Filter\n(edge preserved!)', fontsize=12)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Bilateral filter weights depend on BOTH distance AND intensity!")
print("This preserves edges while smoothing uniform regions.")

### Summary: Mathematical Foundations for Filtering

#### 1. Convolution Theory
- **Definition**: $g(x,y) = \sum_{i,j} f(x+i, y+j) \cdot h(i,j)$
- **Properties**: Commutative, associative, distributive, linear
- **Separability**: 2D convolution = two 1D convolutions (efficiency)

#### 2. Common Kernels
- **Box blur**: Uniform averaging, $h = \frac{1}{n^2}\mathbf{1}$
- **Gaussian**: $G(x,y) = \frac{1}{2\pi\sigma^2}e^{-(x^2+y^2)/(2\sigma^2)}$
- **Sobel**: Gradient approximation, separable
- **Laplacian**: Second derivative, $\nabla^2 f$

#### 3. Non-Linear Filters
- **Median**: Order statistics, robust to outliers
- **Min/Max**: Morphological operations
- **Bilateral**: $w = w_s(\text{distance}) \times w_r(\text{intensity})$

#### 4. Key Properties

| Filter | Linear? | Separable? | Edge-Preserving? | Complexity |
|--------|---------|------------|------------------|------------|
| Box Blur | ✓ | ✓ | ✗ | $O(1)$ (integral image) |
| Gaussian | ✓ | ✓ | ✗ | $O(k)$ separable |
| Sobel | ✓ | ✓ | N/A (edge detector) | $O(k)$ |
| Median | ✗ | ✗ | ✓ | $O(k^2 \log k)$ |
| Bilateral | ✗ | ✗ | ✓ | $O(d^2)$ |

#### Key Formulas Reference

| Operation | Formula | OpenCV Function |
|-----------|---------|----------------|
| Convolution | $g = \sum_{i,j} f(x+i,y+j) \cdot h(i,j)$ | `cv2.filter2D()` |
| Gaussian | $G(x,y) = \frac{1}{2\pi\sigma^2}e^{-(x^2+y^2)/(2\sigma^2)}$ | `cv2.GaussianBlur()` |
| Median | $g = \text{median}\{f(x+i,y+j)\}$ | `cv2.medianBlur()` |
| Bilateral | $w = e^{-d^2/(2\sigma_s^2)} \cdot e^{-\Delta I^2/(2\sigma_r^2)}$ | `cv2.bilateralFilter()` |
| Sobel | $|\nabla f| = \sqrt{(f \ast S_x)^2 + (f \ast S_y)^2}$ | `cv2.Sobel()` |
| Laplacian | $\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}$ | `cv2.Laplacian()` |

---

**Next**: Apply these mathematical foundations to practical filtering and enhancement tasks!

## 3.1 Understanding Convolution

**Convolution** is the fundamental operation in image filtering. It involves sliding a small matrix (kernel/filter) over the image and computing weighted sums.

**Mathematical Definition**:
```
G[i,j] = sum over k,l of: F[i+k, j+l] * K[k,l]
```

Where:
- `F` is the input image
- `K` is the kernel
- `G` is the output image

In [ ]:
# Download sample image
!wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg -O cat.jpg
img = cv2.imread('cat.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(f"Image shape: {img.shape}")
plt.figure(figsize=(8, 6))
plt.imshow(img_rgb)
plt.title('Original Image')
plt.axis('off')
plt.show()

### Simple Box Blur Kernel (Averaging)

In [ ]:
# Create a simple 5x5 averaging kernel
kernel_size = 5
kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size ** 2)

print("Box blur kernel (5x5):")
print(kernel)
print(f"\nSum of kernel: {kernel.sum():.2f} (should be 1.0 for brightness preservation)")

# Apply convolution
blurred = cv2.filter2D(gray, -1, kernel)

# Display
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gray, cmap='gray')
axes[0].set_title('Original Grayscale')
axes[0].axis('off')

axes[1].imshow(blurred, cmap='gray')
axes[1].set_title('Box Blur (5x5)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

### Custom Kernels - Edge Detection

In [ ]:
# Vertical edge detection kernel
kernel_vertical = np.array([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]], dtype=np.float32)

# Horizontal edge detection kernel
kernel_horizontal = np.array([[-1, -2, -1],
                               [ 0,  0,  0],
                               [ 1,  2,  1]], dtype=np.float32)

# Sharpening kernel
kernel_sharpen = np.array([[ 0, -1,  0],
                           [-1,  5, -1],
                           [ 0, -1,  0]], dtype=np.float32)

# Apply kernels
edges_v = cv2.filter2D(gray, -1, kernel_vertical)
edges_h = cv2.filter2D(gray, -1, kernel_horizontal)
sharpened = cv2.filter2D(gray, -1, kernel_sharpen)

# Display results
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(edges_v, cmap='gray')
axes[0, 1].set_title('Vertical Edges')

axes[1, 0].imshow(edges_h, cmap='gray')
axes[1, 0].set_title('Horizontal Edges')

axes[1, 1].imshow(sharpened, cmap='gray')
axes[1, 1].set_title('Sharpened')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 3.2 Gaussian Blur - Smart Smoothing

Gaussian blur uses a weighted average where nearby pixels have more influence. This creates a natural-looking blur.

**Formula**: The kernel follows a Gaussian (bell curve) distribution:
```
G(x, y) = (1/(2*pi*sigma^2)) * exp(-(x^2 + y^2)/(2*sigma^2))
```

**Parameters**:
- Kernel size: Must be odd (3x3, 5x5, 7x7, etc.)
- Sigma (σ): Controls blur strength (larger = more blur)

In [ ]:
# Apply Gaussian blur with different kernel sizes
gaussian_3 = cv2.GaussianBlur(gray, (3, 3), 0)
gaussian_7 = cv2.GaussianBlur(gray, (7, 7), 0)
gaussian_15 = cv2.GaussianBlur(gray, (15, 15), 0)
gaussian_31 = cv2.GaussianBlur(gray, (31, 31), 0)

# Display comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(gaussian_3, cmap='gray')
axes[0, 1].set_title('Gaussian 3x3')

axes[0, 2].imshow(gaussian_7, cmap='gray')
axes[0, 2].set_title('Gaussian 7x7')

axes[1, 0].imshow(gaussian_15, cmap='gray')
axes[1, 0].set_title('Gaussian 15x15')

axes[1, 1].imshow(gaussian_31, cmap='gray')
axes[1, 1].set_title('Gaussian 31x31')

axes[1, 2].axis('off')

for ax in axes.flat[:-1]:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Notice: Larger kernel = stronger blur")

### Different Sigma Values

In [ ]:
# Fixed kernel size, varying sigma
kernel_size = (21, 21)

gaussian_sigma1 = cv2.GaussianBlur(gray, kernel_size, 1)
gaussian_sigma5 = cv2.GaussianBlur(gray, kernel_size, 5)
gaussian_sigma10 = cv2.GaussianBlur(gray, kernel_size, 10)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(gray, cmap='gray')
axes[0].set_title('Original')

axes[1].imshow(gaussian_sigma1, cmap='gray')
axes[1].set_title('Sigma = 1')

axes[2].imshow(gaussian_sigma5, cmap='gray')
axes[2].set_title('Sigma = 5')

axes[3].imshow(gaussian_sigma10, cmap='gray')
axes[3].set_title('Sigma = 10')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 3.3 Median Filter - Noise Removal Champion

**How it works**: Replace each pixel with the **median** value in its neighborhood.

**Best for**: Salt-and-pepper noise (random black/white pixels)

**Advantage**: Preserves edges better than Gaussian blur!

In [ ]:
# Add salt-and-pepper noise
noisy = gray.copy()
noise_ratio = 0.02

# Salt (white pixels)
num_salt = int(noise_ratio * gray.size)
coords = [np.random.randint(0, i, num_salt) for i in gray.shape]
noisy[coords[0], coords[1]] = 255

# Pepper (black pixels)
num_pepper = int(noise_ratio * gray.size)
coords = [np.random.randint(0, i, num_pepper) for i in gray.shape]
noisy[coords[0], coords[1]] = 0

# Apply different filters
gaussian_denoised = cv2.GaussianBlur(noisy, (5, 5), 0)
median_denoised = cv2.medianBlur(noisy, 5)

# Display comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(noisy, cmap='gray')
axes[0, 1].set_title('Noisy (Salt & Pepper)')

axes[1, 0].imshow(gaussian_denoised, cmap='gray')
axes[1, 0].set_title('Gaussian Blur Denoised')

axes[1, 1].imshow(median_denoised, cmap='gray')
axes[1, 1].set_title('Median Filter Denoised (Better!)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Notice: Median filter removes noise while preserving edges better!")

## 3.4 Bilateral Filter - Edge-Preserving Blur

**The Problem**: Gaussian blur smooths everything, including edges!

**The Solution**: Bilateral filter considers **both**:
1. Spatial distance (like Gaussian)
2. Intensity difference (preserves edges)

**Result**: Smooth regions are blurred, but edges stay sharp!

**Parameters**:
- `d`: Diameter of pixel neighborhood
- `sigmaColor`: Color space sigma (larger = more colors mixed)
- `sigmaSpace`: Coordinate space sigma (larger = farther pixels influence)

In [ ]:
# Apply bilateral filter with different parameters
bilateral_9_75_75 = cv2.bilateralFilter(gray, 9, 75, 75)
bilateral_15_80_80 = cv2.bilateralFilter(gray, 15, 80, 80)

# Compare with Gaussian
gaussian_comp = cv2.GaussianBlur(gray, (15, 15), 0)

# Display comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(gaussian_comp, cmap='gray')
axes[0, 1].set_title('Gaussian Blur (edges blurred)')

axes[1, 0].imshow(bilateral_9_75_75, cmap='gray')
axes[1, 0].set_title('Bilateral Filter (d=9)\n(edges preserved!)')

axes[1, 1].imshow(bilateral_15_80_80, cmap='gray')
axes[1, 1].set_title('Bilateral Filter (d=15)\n(stronger smoothing)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Use bilateral filter for:")
print("- Portrait smoothing (skin texture)")
print("- Cartoonization effects")
print("- Edge-aware denoising")

## 3.5 Image Sharpening

**Concept**: Enhance edges to make images appear sharper.

**Method 1**: Unsharp Masking
```
sharpened = original + (original - blurred) * amount
```

**Method 2**: Sharpening kernel (convolution)

In [ ]:
# Method 1: Unsharp Masking
blurred = cv2.GaussianBlur(gray, (9, 9), 10)
unsharp_mask = cv2.addWeighted(gray, 1.5, blurred, -0.5, 0)

# Method 2: Sharpening Kernel
kernel_sharpen = np.array([[ 0, -1,  0],
                           [-1,  5, -1],
                           [ 0, -1,  0]])
kernel_sharpened = cv2.filter2D(gray, -1, kernel_sharpen)

# Strong sharpening kernel
kernel_sharpen_strong = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])
strong_sharpened = cv2.filter2D(gray, -1, kernel_sharpen_strong)

# Display
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(unsharp_mask, cmap='gray')
axes[0, 1].set_title('Unsharp Masking')

axes[1, 0].imshow(kernel_sharpened, cmap='gray')
axes[1, 0].set_title('Kernel Sharpening (Mild)')

axes[1, 1].imshow(strong_sharpened, cmap='gray')
axes[1, 1].set_title('Kernel Sharpening (Strong)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Warning: Over-sharpening can create artifacts!")

## 3.6 Filter Comparison - Which to Use When?

In [ ]:
# Side-by-side comparison on color image
img_blur_gaussian = cv2.GaussianBlur(img, (15, 15), 0)
img_blur_median = cv2.medianBlur(img, 15)
img_blur_bilateral = cv2.bilateralFilter(img, 15, 80, 80)

fig, axes = plt.subplots(2, 2, figsize=(14, 14))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original', fontsize=14)

axes[0, 1].imshow(cv2.cvtColor(img_blur_gaussian, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Gaussian Blur\nFast, smooth everything', fontsize=14)

axes[1, 0].imshow(cv2.cvtColor(img_blur_median, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Median Filter\nBest for salt-pepper noise', fontsize=14)

axes[1, 1].imshow(cv2.cvtColor(img_blur_bilateral, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Bilateral Filter\nEdge-preserving, slower', fontsize=14)

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Filter Selection Guide:")
print("=" * 60)
print("Gaussian Blur: General smoothing, fast")
print("Median Filter: Salt-and-pepper noise removal")
print("Bilateral Filter: Smoothing while preserving edges")
print("Box Blur: Fastest, but lowest quality")

## Project: Build a Photo Filter App

In [ ]:
def apply_vintage_filter(image):
    """Apply vintage/retro filter"""
    # Slight blur for dreamy effect
    img = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Warm color tone (boost red/yellow channels)
    img = img.astype(np.float32)
    img[:, :, 0] *= 0.8  # Reduce blue
    img[:, :, 1] *= 1.1  # Boost green slightly
    img[:, :, 2] *= 1.2  # Boost red
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    return img

def apply_sharp_filter(image):
    """Apply sharpening filter"""
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def apply_cartoon_filter(image):
    """Apply cartoonization effect"""
    # Edge-preserving smoothing
    smooth = cv2.bilateralFilter(image, 9, 75, 75)
    
    # Apply bilateral again for stronger effect
    smooth = cv2.bilateralFilter(smooth, 9, 75, 75)
    
    return smooth

# Apply all filters
vintage = apply_vintage_filter(img)
sharp = apply_sharp_filter(img)
cartoon = apply_cartoon_filter(img)

# Display
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original', fontsize=14)

axes[0, 1].imshow(cv2.cvtColor(vintage, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Vintage Filter', fontsize=14)

axes[1, 0].imshow(cv2.cvtColor(sharp, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Sharp Filter', fontsize=14)

axes[1, 1].imshow(cv2.cvtColor(cartoon, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Cartoon Filter', fontsize=14)

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

# Save filtered images
cv2.imwrite('vintage.jpg', vintage)
cv2.imwrite('sharp.jpg', sharp)
cv2.imwrite('cartoon.jpg', cartoon)
print("Filtered images saved!")

## Summary

You learned:
- **Convolution**: The fundamental operation for image filtering
- **Gaussian Blur**: Natural-looking smoothing with bell curve weights
- **Median Filter**: Best for removing salt-and-pepper noise
- **Bilateral Filter**: Edge-preserving smoothing (cartoonization, portraits)
- **Image Sharpening**: Enhance edges using unsharp masking or kernels

**Key Insights**:
1. Convolution = sliding a kernel over the image
2. Kernel weights determine the effect (blur, sharpen, detect edges)
3. Gaussian preserves brightness (kernel sums to 1)
4. Median is non-linear (no convolution!)
5. Bilateral considers both space AND intensity

**Next**: [Module 4 - Geometric Transformations](04_Module.ipynb)